<a href="https://colab.research.google.com/github/slipaway/melonplaylist/blob/master/final_model_and_result_full_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [3]:
pip install implicit

     |████████████████████████████████| 1.1MB 2.8MB/s 
  Created wheel for implicit: filename=implicit-0.4.2-cp36-cp36m-linux_x86_64.whl size=3420120 sha256=d02d4ff2beaba7f553a7ab9d23dcbd7414c6bd77358abd8b1d9e0abf409ec16c
  Stored in directory: /root/.cache/pip/wheels/1b/48/b1/1aebe3acc3afb5589e72d3e7c3ffc3f637dc4721c1a974dff7
Successfully built implicit


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from datetime import timedelta, datetime
import glob
import os
import re
import seaborn as sns
import warnings
import scipy
warnings.filterwarnings('ignore')
import implicit

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
folder = '/content/gdrive/My Drive/melon kakao'
filelist = os.chdir(folder)

In [6]:
genre_gn_all=pd.read_json('genre_gn_all.json', typ='series')
train=pd.read_json('train.json')
song_meta=pd.read_json('song_meta.json')
test=pd.read_json('test.json')

플레이리스트 생성 시간보다 곡 발매일이 나중에 나온경우 삭제하기

In [7]:
#starnge song 생성

plylst_date = train[['updt_date', 'songs']]
plylst_date_unnest = np.dstack(
    (
        np.repeat(plylst_date.updt_date.values, list(map(len, plylst_date.songs))), 
        np.concatenate(plylst_date.songs.values)
    )
)
plylst_date = pd.DataFrame(data=plylst_date_unnest[0], columns = plylst_date.columns)

del plylst_date_unnest


plylst_date["updt_date"] = plylst_date["updt_date"].map(lambda x: x[:4] + x[5:7] + x[8:10])
song_date = song_meta[["id","issue_date"]]
plylst_song_date = pd.merge(plylst_date, song_date, left_on="songs", right_on="id", how='left')
plylst_song_date["issue_date"] = plylst_song_date["issue_date"].astype(str)

plylst_song_date["strange"] = plylst_song_date["updt_date"] < plylst_song_date["issue_date"]
strange_songs = plylst_song_date[plylst_song_date["strange"] == True].drop('songs', axis=1)
strange_songs=strange_songs.drop_duplicates(['id'])

strange_songs_list=strange_songs['id'].values.astype(str)


#train dataframe 전처리

#플레이리스트 아이디(id)와 수록곡(songs) 추출
plylst_song_map=train[['id', 'songs']]

#unnest songs

plylst_song_map_unnest=np.dstack(
    (
        np.repeat(plylst_song_map.id.values, list(map(len, plylst_song_map.songs))),
        np.concatenate(plylst_song_map.songs.values)
    )
)

#unnested 데이터프레임 생성 : plylst_song_map
plylst_song_map=pd.DataFrame(data=plylst_song_map_unnest[0], columns=plylst_song_map.columns)
plylst_song_map['id']=plylst_song_map['id'].astype(str)
plylst_song_map['songs']=plylst_song_map['songs'].astype(str)

del plylst_song_map_unnest

plylst_song_map_remove=plylst_song_map[np.logical_not(plylst_song_map.songs.isin(strange_songs_list))]


plylst_song_group=plylst_song_map_remove.groupby('id')['songs'].apply(list).reset_index(name='remove_songs')
train['id']=train['id'].astype(str)
train_remove = pd.merge(train, plylst_song_group)
train_remove['songs']=train_remove['remove_songs']
train_remove=train_remove.drop('remove_songs', axis=1)


matrix factorization model 세우기

In [ ]:
#songs 존재하는 test 데이터 남기기

train_songs=train[['id', 'songs']]
test_songs=test[['id', 'songs']]


filter=test_songs['songs'].astype('str')=='[]'
test_songs=test_songs[~filter]

plylst_song_map=pd.concat([train_songs, test_songs])


#plylst_song_map unnest

plylst_song_map_unnest=np.dstack(
    (
        np.repeat(plylst_song_map.id.values, list(map(len, plylst_song_map.songs))),
        np.concatenate(plylst_song_map.songs.values)
    )
)

plylst_song_map=pd.DataFrame(data=plylst_song_map_unnest[0], columns=plylst_song_map.columns)
plylst_song_map['id']=plylst_song_map['id'].astype('category')
plylst_song_map['songs']=plylst_song_map['songs'].astype('category')


In [ ]:
#sparse matrix 만들기

sparse_song=scipy.sparse.coo_matrix((np.ones(plylst_song_map.shape[0]),
                                (plylst_song_map['songs'].cat.codes.copy(),
                                 plylst_song_map['id'].cat.codes.copy())))

In [ ]:
#ALS model 세우기

model_song=implicit.als.AlternatingLeastSquares(factors=10,  regularization=0.01, dtype=np.float64, iterations=20, use_gpu=False)
model_song.fit(sparse_song)

In [ ]:
#recommendation

repos_song=dict(enumerate(plylst_song_map['songs'].cat.categories))

plylst_song_map_test=test_songs

#test set unnest

plylst_song_map_test_unnest=np.dstack(
    (
        np.repeat(plylst_song_map_test.id.values, list(map(len, plylst_song_map_test.songs))),
        np.concatenate(plylst_song_map_test.songs.values)
    )
)

plylst_song_map_test=pd.DataFrame(data=plylst_song_map_test_unnest[0], columns=plylst_song_map_test.columns)
plylst_song_map_test['id']=plylst_song_map_test['id'].astype('category')
plylst_song_map_test['songs']=plylst_song_map_test['songs'].astype('category')

del plylst_song_map_test_unnest

#test_song data에 대한 sparse matrix 만들기

test_song_sparse=scipy.sparse.coo_matrix((np.ones(plylst_song_map_test.shape[0]),
                                          (plylst_song_map_test['id'].cat.codes.copy(),
                                           plylst_song_map_test['songs'].cat.codes.copy()))).tocsr()

In [ ]:
result_songs = []

for i in range(test_songs.shape[0]):
  rec=model_song.recommend(userid=i, user_items=test_song_sparse, N=100)
  result_rec=[]
  for j in range(len(rec)):
    rec[j]=repos_song[rec[j][0]]
    result_rec.insert(j, rec[j])
  result_songs.insert(i, result_rec)


In [ ]:
test_songs['predicted_songs']=result_songs

곡 예측 final - predicted_song

In [ ]:
#곡 prediction 결과

test_result=test_songs.drop('songs', axis=1)
print(test_result)

test_result.to_json('predicted_song.json')

           id                                    predicted_songs
0       70107  [581799, 493762, 15124, 195174, 191915, 146989...
1        7461  [581799, 493762, 15124, 195174, 146989, 191915...
2       90348  [581799, 493762, 15124, 625875, 146989, 195174...
3       58617  [133143, 650494, 701557, 610933, 132994, 34820...
4      102395  [116573, 675115, 654757, 37748, 88503, 663905,...
...       ...                                                ...
10735  137930  [133143, 650494, 701557, 610933, 132994, 34820...
10736     936  [144663, 116573, 357367, 37748, 493762, 627363...
10737  110589  [654757, 117595, 88503, 1236, 113958, 173943, ...
10738    2605  [581799, 493762, 237407, 505036, 15124, 413422...
10739  126651  [519391, 640657, 133484, 698536, 232606, 11228...

[8697 rows x 2 columns]


성환이 곡 예측 데이터 뽑기

In [ ]:
problemset1=pd.read_csv('텅빈애들목록_수정.csv')

filter1=problemset1['songs'].astype('str')=='[]'
problemset1_song=problemset1[~filter1]
problemset1_id=problemset1_song['id']
problemset1_id=problemset1_id.reset_index().drop('index', axis=1)

problem_result_song=[]
for i in range(problemset1_id.shape[0]):
  B=test_result[problemset1_id.iloc[i, 0]==test_result['id']].iloc[0, 1]
  problem_result_song.insert(i, B)


problemset1_id['predicted_songs']=problem_result_song

problemset1_id.to_json('성환노래prediction.json')

In [ ]:
test_result.to_json('predicted_song.json')
problemset1_id.to_json('성환노래prediction.json')

태그 예측하기

In [ ]:
train_tag=train[['id', 'tags']]
test_tag=test[['id', 'tags']]
filter=test_tag['tags'].astype('str')=='[]'
test_tag=test_tag[~filter]


plylst_tag_map=pd.concat([train_tag, test_tag])

#unnesting

plylst_tag_map_unnest=np.dstack(
    (
        np.repeat(plylst_tag_map.id.values, list(map(len, plylst_tag_map.tags))),
        np.concatenate(plylst_tag_map.tags.values)
    )
)

plylst_tag_map=pd.DataFrame(data=plylst_tag_map_unnest[0], columns=plylst_tag_map.columns)
plylst_tag_map['id']=plylst_tag_map['id'].astype('category')
plylst_tag_map['tags']=plylst_tag_map['tags'].astype('category')

del plylst_tag_map_unnest

In [ ]:
#sparse matrix 만들기

sparse_tag=scipy.sparse.coo_matrix((np.ones(plylst_tag_map.shape[0]),
                                    (plylst_tag_map['tags'].cat.codes.copy(),
                                     plylst_tag_map['id'].cat.codes.copy())))

In [ ]:
#모델 세우기

model_tag=implicit.als.AlternatingLeastSquares(factors=20,  regularization=0.01, dtype=np.float64, iterations=20, use_gpu=False)
model_tag.fit(sparse_tag)

In [ ]:
#recommendation

repos_tags=dict(enumerate(plylst_tag_map['tags'].cat.categories))


plylst_tag_map_test=test_tag

#unnesting

plylst_tag_map_test_unnest=np.dstack(
    (
        np.repeat(plylst_tag_map_test.id.values, list(map(len, plylst_tag_map_test.tags))),
        np.concatenate(plylst_tag_map_test.tags.values)
    )
)


plylst_tag_map_test=pd.DataFrame(data=plylst_tag_map_test_unnest[0], columns=plylst_tag_map_test.columns)
plylst_tag_map_test['id']=plylst_tag_map_test['id'].astype('category')
plylst_tag_map_test['tags']=plylst_tag_map_test['tags'].astype('category')


In [ ]:

#test sparse matrix

test_tag_sparse=scipy.sparse.coo_matrix((np.ones(plylst_tag_map_test.shape[0]),
                                          (plylst_tag_map_test['id'].cat.codes.copy(),
                                           plylst_tag_map_test['tags'].cat.codes.copy()))).tocsr()

In [ ]:
result_tag=[]

for i in range(test_tag.shape[0]):
  rec=model_tag.recommend(i, test_tag_sparse, N=10)
  result_rec=[]
  for j in range(len(rec)):
    rec[j]=repos_tags[rec[j][0]]
    result_rec.insert(j, rec[j])
  result_tag.insert(i, result_rec)


In [ ]:
test_result_tag=test_tag
test_result_tag['predicted_tags']=result_tag
test_result_tag=test_result_tag.drop('tags', axis=1).reset_index()

In [ ]:
test_result_tag.to_json('predicted_tag.json')

성환이 데이터에 태그 채우기

In [ ]:
problemset1

filter4=problemset1['tags'].astype('str')=='[]'
problemset1_tag=problemset1[~filter4]
problemset1_tag_id=problemset1_tag['id'].reset_index().drop('index', axis=1)

print(problemset1_tag_id)

        id
0    81933
1   139673
2   119362
3   129084
4     7713
5    76637
6   106582
7   119817
8     8572
9    37362
10   68319
11   90892
12  102824
13   97369
14   42898
15   54496


In [ ]:
problem_result_tag=[]
for i in range(problemset1_tag_id.shape[0]):
  B=test_result_tag[(problemset1_tag_id.iloc[i, 0]==test_result_tag['id']).tolist()].iloc[0, 2]
  problem_result_tag.insert(i, B)


In [ ]:
problemset1_tag_id['predicted_tag']=problem_result_tag

In [ ]:
problemset1_tag_id

,id,predicted_tag
0,81933,"[휴식, 드라이브, 힐링, 스트레스, Pop, 운동, 저녁, 주말, 분위기, 트렌디]"
1,139673,"[힙합, 알앤비, 랩, 소울, 락, 매장음악, RnB, 그루브, 트렌디, 가을]"
2,119362,"[신나는, 여행, 댄스, 스트레스, 여름, 운동, 드라이브, EDM, 클럽, 산책]"
3,129084,"[알앤비, 소울, 락, 가을, RnB, 그루브, 재즈, 트렌디, 카페, 분위기]"
4,7713,"[힙합, 랩, 알앤비, HipHop, 국내힙합, 트렌디, 감성힙합, 외힙, 클럽, ..."
5,76637,"[힙합, 랩, 겨울, HipHop, 국내힙합, 외힙, 감성힙합, 국힙, 외국힙합, ..."
6,106582,"[감성, 신나는, 잔잔한, 여행, 카페, 가을, 댄스, 여름, 스트레스, 알앤비]"
7,119817,"[알앤비, 소울, 락, 가을, 힙합, RnB, 그루브, 카페, 트렌디, 분위기]"
8,8572,"[힙합, 랩, 알앤비, HipHop, 국내힙합, 트렌디, 감성힙합, 외힙, 클럽, ..."
9,37362,"[알앤비, 소울, 락, 가을, 힙합, RnB, 그루브, 카페, 트렌디, 분위기]"


In [ ]:
problemset1_song_id=problemset1_id
print(problemset1_song_id)
print(problemset1_tag_id)

problemset1_song_id.to_json('빈노래 채우기.json')
problemset1_tag_id.to_json('빈태그 채우기.json')

         id                                    predicted_songs
0     51734  [650494, 133143, 701557, 610933, 132994, 34820...
1    145709  [116573, 133143, 144663, 650494, 610933, 70155...
2    135315  [581799, 493762, 625875, 15124, 146989, 464051...
3     93364  [581799, 493762, 15124, 195174, 146989, 625875...
4     48871  [696494, 202185, 571790, 26590, 52192, 224568,...
..      ...                                                ...
204  107623  [650494, 133143, 701557, 348200, 610933, 13299...
205  115081  [144663, 357367, 627363, 174749, 519391, 64065...
206   81238  [581799, 493762, 15124, 195174, 146989, 191915...
207   93936  [581799, 493762, 15124, 195174, 191915, 146989...
208  122238  [581799, 493762, 15124, 195174, 191915, 146989...

[209 rows x 2 columns]
        id                                      predicted_tag
0    81933    [휴식, 드라이브, 힐링, 스트레스, Pop, 운동, 저녁, 주말, 분위기, 트렌디]
1   139673       [힙합, 알앤비, 랩, 소울, 락, 매장음악, RnB, 그루브, 트렌디, 가을]
2   119362     [신나는, 여행, 댄스, 스트레스,

In [ ]:
problemset1_tag_id.to_json('빈태그 채우기.json')

In [ ]:
problemset1

#태그가 있는 것 : 15개
#노래가 있는 것 : 209개
482-224=258개 - 256개는 제목 있고, 2개 : 아예 없음

,Unnamed: 0,tags,id,plylst_title,songs,like_cnt,updt_date
0,0,[],51734,NaN,"[635091, 146442, 465849, 357297, 261979, 25677...",10,2011-10-01 07:22:09.000
1,1,[],145709,NaN,"[280018, 528539, 605842, 378826, 137408, 25855...",2176,2010-01-08 17:52:52.000
2,2,[],135315,NaN,"[396168, 317527, 16181, 346252, 334818]",6,2013-05-18 08:14:32.000
3,3,[],93364,NaN,"[421963, 159925, 233079, 221309, 578190, 538918]",27,2018-09-15 13:17:20.000
4,4,[],48871,NaN,"[620694, 238475, 371335, 54921, 172903, 146043...",106,2019-04-16 10:08:02.000
...,...,...,...,...,...,...,...
477,477,[],46271,"따뜻한 위로, 그리고 힘찬 응원!",[],170,2016-03-29 17:08:42.000
478,478,[],81238,NaN,"[118744, 30289, 426529, 702921, 308866, 655637...",3,2016-01-22 19:19:00.000
479,479,[],93936,NaN,"[129269, 75236, 411494, 169006, 538348, 21888,...",17,2020-01-03 17:22:02.000
480,480,[],114894,신나게씽씽,[],12,2019-06-25 08:01:39.000


In [ ]:
problemset1[~filter4]

,Unnamed: 0,tags,id,plylst_title,songs,like_cnt,updt_date
19,19,['봄햇살에'],81933,공간에 설레임을 입히다! Made My Heart Flutter,[],11,2016-11-17 13:14:39.000
33,33,['이너피스'],139673,테스트 신청,[],0,2019-04-29 18:53:03.000
60,60,['헬로카봇'],119362,헬로카봇! 신나게 들어요!,[],45,2020-02-03 18:07:05.000
110,110,['thepuplealbum'],129084,Lukas Graham,[],36,2020-03-20 22:49:23.000
116,116,"['미자탈출', '오늘부터성인']",7713,경축 미자탈출! 올해 스무살이 된 아티스트의 생기 가득한 곡,[],96,2020-04-02 12:34:31.000
156,156,['이야'],76637,호호호호홓,[],13,2020-02-23 18:11:33.000
170,170,['누구나'],106582,누구나 쉽게 즐길 수 있는 신나는 락음악!,[],3,2016-09-07 07:27:24.000
175,175,['그어놓은'],119817,따뜻한 동행,[],3,2020-02-29 04:45:42.000
191,191,['인트로맛집'],8572,여기 인트로 맛집이네^^...,[],2,2020-03-18 15:18:53.000
250,250,['꽃보다청춘아프리카편'],37362,꽃보다청춘 1회(예고편포함) 브금,[],11,2016-02-23 18:39:24.000


In [ ]:
# raw 데이터 : 태그 있으면 제목 있고, 노래 없음
# raw 데이터 : 노래 있으면 태그 제목 둘다 없음
# 제목만 있는 경우

현주 빈 태그 채워넣기

In [ ]:
problemset2=pd.read_csv('현주안되는거.csv')
problemset2_id=problemset2['id']


0        9823
1       31103
2       75837
3       81933
4       95606
        ...  
149     27708
150     30510
151    115035
152     77836
153     27739
Name: id, Length: 154, dtype: int64

In [ ]:
test_result_tag[test_result_tag['id']==9823].iloc[0, 1]

['힙합', '랩', '알앤비', '소울', 'HipHop', '가을', '트렌디', '그루브', '국내힙합', '카페']

In [ ]:
problemset2_tag=[]
for i in range(problemset2_id.shape[0]-1):
  B=test_result_tag[problemset2_id[i]==test_result_tag['id']].iloc[0, 1]
  problemset2_tag.insert(i, B)

In [ ]:
problemset2['predicted_tags']=problemset2_tag

In [ ]:
problemset2_result=problemset2[['id', 'predicted_tags']]

In [ ]:
problemset2_result.to_json('현주결과못내는tag.json')

성환+현주+재현 case

In [10]:
test_hyunju=pd.read_json('성환오빠 채운거.json')

test_id = test['id'].tolist()
test_h_id = test_hyunju['id'].tolist()
len_1 = test.shape[0]
len_2 = test_hyunju.shape[0]

for i in range(len_1) :
  for j in range(len_2) :
    if test_id[i] == test_h_id[j]:
      test['tags'][i] = test_hyunju['predicted_tags'][j]

8697

In [ ]:
train_tag=train[['id', 'tags']]
test_tag=test[['id', 'tags']]

plylst_tag_map=pd.concat([train_tag, test_tag])

plylst_tag_map_unnest=np.dstack(
    (
        np.repeat(plylst_tag_map.id.values, list(map(len, plylst_tag_map.tags))),
        np.concatenate(plylst_tag_map.tags.values)
    )
)

plylst_tag_map=pd.DataFrame(data=plylst_tag_map_unnest[0], columns=plylst_tag_map.columns)
plylst_tag_map['id']=plylst_tag_map['id'].astype('category')
plylst_tag_map['tags']=plylst_tag_map['tags'].astype('category')

del plylst_tag_map_unnest

In [ ]:
plylst_tag_map['id']=plylst_tag_map['id'].astype('category')
plylst_tag_map['tags']=plylst_tag_map['tags'].astype('category')

tag_sparse=scipy.sparse.coo_matrix((np.ones(plylst_tag_map.shape[0]),
                                       (plylst_tag_map['tags'].cat.codes.copy(), 
                                        plylst_tag_map['id'].cat.codes.copy())))

In [ ]:
model_tag_col=implicit.als.AlternatingLeastSquares(factors=30, regularization=0.005, dtype=np.float64, iterations=40, use_gpu=False)
model_tag_col.fit(tag_sparse)

In [ ]:
repos_tag=dict(enumerate(plylst_tag_map['tags'].cat.categories))

filter1=test_hyunju['predicted_tags'].astype('str')=='[]'
test_hyunju=test_hyunju[~filter1]

plylst_tag_map_test=test_hyunju[['id', 'predicted_tags']]
plylst_tag_map_test.columns=['id', 'tags']

plylst_tag_map_test_unnest=np.dstack(
    (
        np.repeat(plylst_tag_map_test.id.values, list(map(len, plylst_tag_map_test.tags))),
        np.concatenate(plylst_tag_map_test.tags.values)
    )
)

plylst_tag_map_test=pd.DataFrame(data=plylst_tag_map_test_unnest[0], columns=plylst_tag_map.columns)

del(plylst_tag_map_test_unnest)

In [ ]:
plylst_tag_map_test['id']=plylst_tag_map_test['id'].astype('category')
plylst_tag_map_test['tags']=plylst_tag_map_test['tags'].astype('category')
test_sparse=scipy.sparse.coo_matrix((np.ones(plylst_tag_map_test.shape[0]),
                                       (plylst_tag_map_test['id'].cat.codes.copy(), 
                                        plylst_tag_map_test['tags'].cat.codes.copy()))).tocsr()

In [ ]:
result_tag=[]
for i in range(test_hyunju.shape[0]):
  B=model_tag_col.recommend(userid=i, user_items=test_sparse, N=10)
  result_B=[]
  for j in range(len(B)):
    B[j]=repos_tag[B[j][0]]
    result_B.insert(j, B[j])

  result_tag.insert(i, result_B)


In [ ]:
new_result=[]
for i in range(test_hyunju.shape[0]):
  test_hyunju['predicted_tags'][i].extend(result_tag[i])
  new_result.insert(i, test_hyunju['predicted_tags'][i][:10])

In [ ]:
test_hyunju['new']=new_result

collabo_result=test_hyunju[['id', 'new']]
collabo_result.columns=['id', 'predicted_tags']

In [ ]:
filter2=test1['predicted_tags'].astype('str')=='[]'
test_nodata=test1[filter2]

collabo_result=pd.concat([collabo_result, test_nodata])
collabo_result.columns=['id', 'predicted_tags']
collabo_result.reset_index(inplace=True)
collabo_result=collabo_result.drop('index', axis=1)

collabo_result.to_json('성환+현주+재현.json')